# ▶️ CUDA setup

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!nvidia-smi

Thu Dec  5 19:08:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

GPU computing notebooks download (from github)

In [ ]:
!git clone https://github.com/giulianogrossi/GPUcomputing.git

Cloning into 'GPUcomputing'...
remote: Enumerating objects: 575, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 575 (delta 100), reused 214 (delta 98), pack-reused 347 (from 1)
Receiving objects: 100% (575/575), 2.79 MiB | 16.31 MiB/s, done.
Resolving deltas: 100% (257/257), done.


NVCC Plugin for Jupyter notebook

In [ ]:
%cd GPUcomputing/utils/nvcc4jupyter-master/
!!python3 -m build
%load_ext nvcc4jupyter
%cd /content/

/content/GPUcomputing/utils/nvcc4jupyter-master
Source files will be saved in "./src".
/content


# ▶️ Device Query

In [ ]:
# DeviceQuery dell'attuale device (su Colab!)
!nvcc -arch=sm_75 /content/GPUcomputing/utils/deviceQuery.cu -o deviceQuery
!./deviceQuery


CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          12.2 / 12.2
  GPU arch name:                                 Turing
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15102 MBytes (15835660288 bytes)
  (40) Multiprocessors, ( 64) CUDA Cores/MP:     2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory                65536

#  ✅ Confronto algoritmi


In [ ]:
%%cuda_group_save --name main.cu --group "progetto_GPU"
#include "sequential_dijkstra.h"
#include "sourcePartitioned.h"

#include <stdio.h>
#include <limits.h>
#include <cuda_runtime.h>
#include "/content/GPUcomputing/utils/common.h"

#define INF 999
#define N 1024

int main () {

    double start, stopPartitoned, stopParallel, stopCPU, speedupPartitioned,speedupParallel;
    /*alloco la memoria dinamicamente, la matrice è lunga NxN , rappresentata linearmente*/
    int* matrice = (int*)malloc(N * N * sizeof(int));

    /*Verifica l allocazione della memoria*/
    if (matrice == NULL) {
        printf("Errore di allocazione della memoria\n");
        exit(1);
    }

     for (int i = 0; i < N; i++) {
        matrice[i] = INF;
     }

    randomGraph(matrice, N);

    printf("Random graph of %d nodes initialized\n", N);
    /*printGraph(matrice, N)*/;

    /*alloco matrice dei risultati su CPU per il partitioned*/
    int* resultsPartitioned = (int*)malloc(N * N * sizeof(int));
    int* resultsParallel = (int*)malloc(N * N * sizeof(int));

    /*alloco matrice della GPU*/
    int* gpu_matrix;
    cudaError_t cudaError = cudaMalloc(&gpu_matrix, N * N * sizeof(int));

    if (cudaError != cudaSuccess) {
        printf("Errore during matrix allocation on GPU: %s\n", cudaGetErrorString(cudaError));
        exit(1);
    }
    printf("Matrix allocation on GPU completed\n");

    /*copio matrice da CPU a GPU*/
    cudaError = cudaMemcpy(gpu_matrix, matrice, N * N * sizeof(int), cudaMemcpyHostToDevice);
    if (cudaError != cudaSuccess) {
        printf("Error during matrix copy on GPU: %s\n", cudaGetErrorString(cudaError));
        exit(1);
    }
    printf("Matrix copy on GPU completed\n");


    /*per memorizzare i risultati per il sequenziale*/
    int* results = (int*)malloc(N * N * sizeof(int));

    /*per memorizzare i risultati su GPU*/
    int* resultsMatrix;
    cudaError = cudaMalloc(&resultsMatrix, N * N * sizeof(int));

    if (cudaError != cudaSuccess) {
        printf("Error during results matrix allocation on GPU: %s\n", cudaGetErrorString(cudaError));
        exit(1);
    }
    printf("Results matrix allocation completed\n");

    /*array l su GPU*/
    int* lArray;
    cudaError = cudaMalloc(&lArray, N * N * sizeof(int));

    if (cudaError != cudaSuccess) {
        printf("Error during results lArray allocation on GPU: %s\n", cudaGetErrorString(cudaError));
        exit(1);
    }
    printf("lArray allocation completed\n");

    /*nodi visitati su GPU*/
    bool* VtArray;
    cudaError = cudaMalloc(&VtArray, N * N * sizeof(bool));

    if (cudaError != cudaSuccess) {
        printf("Error during results VtArray allocation on GPU: %s\n", cudaGetErrorString(cudaError));
        exit(1);
    }
    printf("VtArray allocation completed\n");


    printf("Matrice di adiacenza del grafo casuale:\n");

    for (int i = 0; i < N * N; i++) {
        results[i] = 0;
    }

    start = seconds();
    dijkstraCPU((int*)matrice, N,(int*)results);
    stopCPU = seconds() - start;
    printf("Tempo di esecuzione CPU: %f\n",stopCPU);
    free(matrice);

    printf("\n\nSOURCE PARTITONED PART\n\n");

    int threadsPerBlock = 1024;
    int blocks = (N + threadsPerBlock - 1) / threadsPerBlock;
    start = seconds();
    sourcePartitioned<<< blocks, threadsPerBlock >>> (gpu_matrix, N, resultsMatrix, lArray, VtArray);

    cudaError = cudaGetLastError();

    if (cudaError != cudaSuccess) {
        printf("Error during kernel SOURCE PARTITONED launch: %s\n", cudaGetErrorString(cudaError));
        exit(1);
    }

    cudaError = cudaDeviceSynchronize();
    if (cudaError != cudaSuccess) {
        printf("Kernel syncronization returned error: %s\n", cudaGetErrorString(cudaError));
        exit(1);
    }
    stopPartitoned = seconds() - start;
    printf("Tempo di esecuzione Source Partitioned: %f\n",stopPartitoned);

    cudaError = cudaMemcpy(results, resultsMatrix, N * N * sizeof(int), cudaMemcpyDeviceToHost);
    if (cudaError != cudaSuccess) {
        printf("Error during results copy on Host: %s\n", cudaGetErrorString(cudaError));
    }
    printf("Results copy on Host completed\n");

    for (int i = 0; i < N * N; i++) {
        resultsPartitioned[i] = results[i];
    }

    cudaFree(lArray);
    cudaFree(VtArray);



    if (N <= 1024) {
        printf("\n\nSOURCE PARALLEL\n\n");

        // Inizializzo di nuovo la matrice dei risultati
        for (int i = 0; i < N * N; i++) {
            results[i] = 0;
        }

        size_t sharedMemSize = sizeof(int) * N * 3 + sizeof(bool) * (N + 1);

        start=seconds();
        sourceParallel << <N, N, sharedMemSize >> > (gpu_matrix, resultsMatrix);

        cudaError = cudaGetLastError();

        if (cudaError != cudaSuccess) {
            printf("Error during kernel SOURCE PARALLEL launch: %s\n", cudaGetErrorString(cudaError));
            exit(1);
        }

        cudaError = cudaPeekAtLastError();
        if (cudaError != cudaSuccess) {
            printf("Error during kernel SOURCE PARALLEL execution: %s\n", cudaGetErrorString(cudaError));
            exit(1);
        }

        cudaError = cudaDeviceSynchronize();
        if (cudaError != cudaSuccess) {
            printf("Kernel SOURCE PARALLEL syncronization returned error: %s\n", cudaGetErrorString(cudaError));
            exit(1);
        }

        stopParallel = seconds() - start;
        printf("Tempo di esecuzione Source Parallel: %f\n",stopParallel);

        cudaError = cudaMemcpy(results, resultsMatrix, N * N * sizeof(int), cudaMemcpyDeviceToHost);
        if (cudaError != cudaSuccess) {
            printf("Error during results copy on Host: %s\n", cudaGetErrorString(cudaError));
        }
        printf("Results copy on Host completed\n");

        for (int i = 0; i < N * N; i++) {
            resultsParallel[i] = results[i];
        }

    }

    speedupPartitioned = stopCPU / stopPartitoned;
    printf("Speedup Partitoned: %f\n", speedupPartitioned);

    speedupParallel = stopCPU / stopParallel;
    printf("Speedup Parallel: %f\n", speedupParallel);

    cudaFree(resultsMatrix);
    cudaFree(gpu_matrix);

    free(results);
    free(matrice);
    return 0;
}


# ✅ Algoritmo sequenziale Dijkstra


In [ ]:
%%cuda_group_save --name sequential_dijkstra.cu --group "progetto_GPU"
#include "sequential_dijkstra.h"

#include <stdio.h>
#include <limits.h>
#include <cuda_runtime.h>
#include <time.h>

#define INF 9999



bool tuttiVeri(bool* vector, int dimension) {
    for (int i = 0; i < dimension; i++) {
        if (vector[i] == false) return false;
    }
    return true;
}


int getRandomWeight() {
    /*pesi possibili grafo*/
    int list[] = { 1, 2, 3, 4, 5, INF };
    int index = rand() % (sizeof(list) / sizeof(list[0]));
    return list[index];
}


void dijkstraCPU(int* matrice, int n, int* results) {
        printf("CPU\n");

    for (int nodo = 0; nodo < n; nodo++) {
      int distanza[n];
      bool V_t[n];
      for (int i = 0; i < n; i++) {
          V_t[i] = false;
      }
      V_t[nodo] = true;


      for (int i = 0; i < n; i++) {
          distanza[i] = matrice[nodo * n + i];
      }

    while (tuttiVeri(V_t, n) == false) {
            int minDist = INF;
            int minIdx = nodo;

            /*trovo il nodo non visitato con la distanza minima dalla sorgente*/
            for (int i = 0; i < n; i++) {
                if (V_t[i] == false && distanza[i] < minDist) {
                    minDist = distanza[i];
                    minIdx = i;
                }
            }
           V_t[minIdx] = true;

           /*ricalcolo distanze per vertici non in V_t*/
           for (int j = 0; j < n; j++) {
              if (V_t[j] == false && matrice[minIdx * n + j] != 0 && distanza[minIdx] != INF && distanza[minIdx] + matrice[minIdx * n + j] < distanza[j])
                  distanza[j] = distanza[minIdx] + matrice[minIdx * n + j];
                    }
    }
    /*
      printf("Distanze dal nodo %d:\n", nodo);
    for (int i = 0; i < n; i++) {
      if (distanza[i] == INF)
          printf("Nodo %d: INFINITO\n", i);
      else
          printf("Nodo %d: %d\n", i, distanza[i]);
      }
    */

     for (int i = 0; i < n; i++) {
            results[nodo * n + i] = distanza[i];
        }

    }

  }



void randomGraph(int* matrix, int dimension) {
    srand(time(NULL));
    /* i è la riga,j la colonna*/
    for (int i = 0; i < dimension; i++) {
        for (int j = 0; j < dimension; j++) {
            if (i == j) {
                matrix[i * dimension + j] = 0;
            } else {
                int weight = getRandomWeight();
                matrix[i * dimension + j] = weight;
                matrix[j * dimension + i] = weight;
            }
        }
    }
}


void printGraph(int* matrix, int dimension) {
    for (int i = 0; i < dimension; i++) {
        for (int j = 0; j < dimension; j++) {
            if (matrix[i * dimension + j] == INF) {
                printf("INF ");
            } else {
                printf("%d ", matrix[i * dimension + j]);
            }
        }
        printf("\n");
    }
}



In [ ]:
%%cuda_group_save --name sequential_dijkstra.h --group "progetto_GPU"

using namespace std;

extern bool tuttiVeri(bool* vector, int dimension);

extern void dijkstraCPU(int* matrice, int n, int* results);
extern void randomGraph(int* matrix, int dimension);
extern void printGraph(int* matrix, int dimension);

In [ ]:
!nvcc -arch=sm_75 src/progetto_GPU/sequential_dijkstra.cu -o sequential_dijkstra
!./sequential_dijkstra

/usr/bin/ld: /usr/lib/gcc/x86_64-linux-gnu/11/../../../x86_64-linux-gnu/Scrt1.o: in function `_start':
(.text+0x1b): undefined reference to `main'
collect2: error: ld returned 1 exit status
/bin/bash: line 1: ./sequential_dijkstra: No such file or directory



# ✅ Algoritmo Source Partitioned e Source Parallel Dijkstra

In [ ]:
%%cuda_group_save --name sourcePartitioned.cu --group "progetto_GPU"
#include "sourcePartitioned.h"

#include <stdio.h>
#include <limits.h>
#include <cuda_runtime.h>

#define INF 999
#define INF 999

#ifdef __CUDACC__
#define __syncthreads() __syncthreads()
#else
#define __syncthreads()
#endif

__device__ int minimoIndiceVt(int a, int b, int nodeIndexA, int nodeIndexB, bool VtA, bool VtB) {
    if (VtA && VtB == false) return nodeIndexB;
    if (VtB && VtA == false) return nodeIndexA;
    int minimum = min(a, b);
    return minimum == a ? nodeIndexA : nodeIndexB;
}

__device__ int minimoVt(int a, int b, bool VtA, bool VtB) {
    if (VtA && VtB == false) return b;
    if (VtB && VtA == false) return a;
    return min(a, b);
}

__device__ bool tuttiVeriGPU(bool* vector, int dimension) {
    for (int i = 0; i < dimension; i++) {
        if (vector[i] == false) return false;
    }
    return true;
}

__global__ void sourcePartitioned(int* matrice, int dimension, int* results, int* lArray, bool* VtArray) {


    /*Trovo l indice globale del thread*/
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < dimension) {

        /*prendo solo la parte di VtArray riguardante il mio nodo, punto l indirizzo che ha idx*dimension posizioni dopo l inizio di VtArray*/
        bool* V_t = VtArray + idx * dimension;
        /*inizializzo V_t*/
        for (int i = 0; i < dimension; i++) {
                V_t[i] = false;
            }
        V_t[idx] = true;

        /*prendo la parte di lArray riguardante il nodo corrente*/
        int* distanze = lArray + idx * dimension;
        /*inizializzo il vettore delle distanze*/
        for (int i = 0; i < dimension; i++) {
            distanze[i] = matrice[idx * dimension + i];
        }

        while (tuttiVeriGPU(V_t, dimension) == false) {

            int minDist = INF;
            int minIdx = idx;

            /*trovo il nodo non visitato con la distanza minima dalla sorgente*/
            for (int i = 0; i < dimension; i++) {
                if (V_t[i] == false && distanze[i] < minDist) {
                    minDist = distanze[i];
                    minIdx = i;
                }
            }

            /* aggiungo a V_t il nodo piu vicino*/
            V_t[minIdx] = true;

            /*ricalcolo distanze per vertici non in V_t*/
           for (int j = 0; j < dimension; j++) {
              if (V_t[j] == false && matrice[minIdx * dimension + j] != 0 && distanze[minIdx] != INF && distanze[minIdx] + matrice[minIdx * dimension + j] < distanze[j])
                  distanze[j] = distanze[minIdx] + matrice[minIdx * dimension + j];
                    }
        }

        /*
         if(idx==0){
          printf("GPU\n");

          printf("Distanze dal nodo %d:\n", idx);

          for (int i = 0; i < dimension; i++) {
            if (distanze[i] == INF)
              printf("Nodo %d: INFINITO\n", i);
            else
              printf("Nodo %d: %d\n", i, distanze[i]);
            }

         }*/


        for (int i = 0; i < dimension; i++) {
            results[idx * dimension + i] = distanze[i];
        }
    }
}

__global__ void sourceParallel(int* matrix, int* results) {
    int tID = threadIdx.x;
    int bID = blockIdx.x;
    int bDim = blockDim.x;

    extern __shared__ int s[];
    int* sharedData = s;

    int* distanze = (int*)&sharedData[0];


    int* valoriMinimi = (int*)&distanze[bDim];
    int* indiciMinimi = (int*)&valoriMinimi[bDim];

    bool* V_t = (bool*)&indiciMinimi[bDim];

    bool* conclusione = (bool*)&V_t[bDim];

    V_t[tID] = false;

    if (tID == 0) {
        V_t[bID] = true;
    }
    __syncthreads();

    distanze[tID] = matrix[bID * bDim + tID];

    __syncthreads();

    while (true) {

        if (tID == 0) {
            conclusione[0] = tuttiVeriGPU(V_t, bDim);
        }
        __syncthreads();

        if (conclusione[0]) {
            break;
        }
        __syncthreads();

        valoriMinimi[tID] = distanze[tID];
        indiciMinimi[tID] = tID;

        for (int stride = 1; stride < bDim; stride *= 2) {
            int index = 2 * stride * tID;

            if (index + stride < bDim) {

                int minimoLocale = minimoVt(
                    valoriMinimi[index],
                    valoriMinimi[index + stride],
                    V_t[indiciMinimi[index]],
                    V_t[indiciMinimi[index + stride]]
                );

                int indiceMinimoLocale = minimoIndiceVt(
                    valoriMinimi[index],
                    valoriMinimi[index + stride],
                    indiciMinimi[index],
                    indiciMinimi[index + stride],
                    V_t[indiciMinimi[index]],
                    V_t[indiciMinimi[index + stride]]
                );

                indiciMinimi[index] = indiceMinimoLocale;
                valoriMinimi[index] = minimoLocale;

            }
            __syncthreads();
        }

        if (tID == 0) {
            V_t[indiciMinimi[0]] = true;
        }
        __syncthreads();

        if (V_t[tID]== false) {

            int pesoUv = matrix[indiciMinimi[0] * bDim + tID];
            distanze[tID] = min(distanze[tID], distanze[indiciMinimi[0]] + pesoUv);

        }


        __syncthreads();

    }

/*
if (tID == 0) {
    printf("Iterazione iniziale, distanze dal nodo %d:\n", bID);  // bID rappresenta il nodo di partenza
    for (int i = 0; i < bDim; i++) {
        if (distanze[i] == INF) {
            printf("Nodo %d -> Nodo %d: INFINITO\n", bID, i);  // Nodo di partenza bID verso nodo i
        } else {
            printf("Nodo %d -> Nodo %d: %d\n", bID, i, distanze[i]);  // Nodo di partenza bID verso nodo i con distanza
        }
    }
    printf("\n");
}
*/

    results[bID * bDim + tID] = distanze[tID];
}


In [ ]:
%%cuda_group_save --name sourcePartitioned.h --group "progetto_GPU"

using namespace std;

__device__ extern bool tuttiVeriGPU(bool* vector, int dimension);

__device__ int minimoIndiceVt(int a, int b, int nodeIndexA, int nodeIndexB, bool VtA, bool VtB);

__device__ int minimoVt(int a, int b, bool VtA, bool VtB);

__global__ extern void sourcePartitioned(int* matrice, int dimension, int* results, int* lArray, bool* VtArray);

__global__ void sourceParallel(int* matrix, int* results);



In [ ]:
!nvcc -arch=sm_75 src/progetto_GPU/sourcePartitioned.cu main.cu -o sourcePartitioned
!./sourcePartitioned

cc1plus: fatal error: main.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./sourcePartitioned: No such file or directory


# Esecuzione

In [ ]:
!nvcc -arch=sm_75 -o main src/progetto_GPU/main.cu src/progetto_GPU/sequential_dijkstra.cu src/progetto_GPU/sourcePartitioned.cu
!./main


Random graph of 1024 nodes initialized
Matrix allocation on GPU completed
Matrix copy on GPU completed
Results matrix allocation completed
lArray allocation completed
VtArray allocation completed
Matrice di adiacenza del grafo casuale:
CPU
Tempo di esecuzione CPU: 9.172480


SOURCE PARTITONED PART

Tempo di esecuzione Source Partitioned: 3.614423
Results copy on Host completed


SOURCE PARALLEL

Tempo di esecuzione Source Parallel: 0.610411
Results copy on Host completed
Speedup Partitoned: 2.537744
Speedup Parallel: 15.026724


In [ ]:
!ncu ./main


Random graph of 2048 nodes initialized
==PROF== Connected to process 16454 (/content/main)
Matrix allocation on GPU completed
Matrix copy on GPU completed
Results matrix allocation completed
lArray allocation completed
VtArray allocation completed
Matrice di adiacenza del grafo casuale:
CPU
==PROF== Received signal
==PROF== Trying to shutdown target application
==WARNING== No kernels were profiled.
==WARNING== Profiling kernels launched by child processes requires the --target-processes all option.
